In [2]:
from acquire import get_telco_data

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder


In [3]:
df = get_telco_data()

choices_lns = [2, 1, 0]
choices = [3, 2, 1, 0]

conditions_secback = [
    (df['online_security'] == 'Yes') & (df['online_backup'] == 'Yes'),
    (df['online_security'] == 'No') & (df['online_backup'] == 'Yes'), 
    (df['online_security'] == 'Yes') & (df['online_backup'] == 'No'),
    (df['online_security'] == 'No') & (df['online_backup'] == 'No')
    ]

conditions_strm = [
    (df['streaming_tv'] == 'Yes') & (df['streaming_movies'] == 'Yes'),
    (df['streaming_tv'] == 'No') & (df['streaming_movies'] == 'Yes'), 
    (df['streaming_tv'] == 'Yes') & (df['streaming_movies'] == 'No'),
    (df['streaming_tv'] == 'No') & (df['streaming_movies'] == 'No')
]

conditions_pdep = [
    (df['partner'] == 'Yes') & (df['dependents'] == 'Yes'),
    (df['partner'] == 'No') & (df['dependents'] == 'Yes'), 
    (df['partner'] == 'Yes') & (df['dependents'] == 'No'),
    (df['partner'] == 'No') & (df['dependents'] == 'No')
]

conditions_lns = [
    (df['phone_service'] == 'Yes') & (df['multiple_lines'] == 'Yes'),
    (df['phone_service'] == 'Yes') & (df['multiple_lines'] == 'No'),
    (df['phone_service'] == 'No') & (df['multiple_lines'] == 'No')
]

def conditional_encodes(df):
    df['num_lines'] = np.select(conditions_lns, choices_lns)
    df['partner_dependents'] = np.select(conditions_pdep, choices)
    df['streaming_services'] = np.select(conditions_strm, choices)
    df['secback'] = np.select(conditions_secback, choices)
    return df


df = conditional_encodes(df)
df.head()


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,num_lines,partner_dependents,streaming_services,secback
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,59.9,542.4,No,Month-to-month,DSL,Mailed check,2,0,2,0
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic),1,2,3,0
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,48.2,340.35,No,Month-to-month,DSL,Electronic check,1,0,0,1
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,No,...,25.1,25.1,Yes,Month-to-month,DSL,Electronic check,0,0,0,0
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,1,Yes,...,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic),0,3,0,1


In [4]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,num_lines,partner_dependents,streaming_services,secback
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,59.9,542.4,No,Month-to-month,DSL,Mailed check,2,0,2,0
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic),1,2,3,0
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,48.2,340.35,No,Month-to-month,DSL,Electronic check,1,0,0,1
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,No,...,25.1,25.1,Yes,Month-to-month,DSL,Electronic check,0,0,0,0
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,1,Yes,...,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic),0,3,0,1


In [5]:
def churn_num(dataframe):
    return dataframe.assign(
        churn = dataframe['churn'].map({'No': 0, 'Yes': 1})

    )
df = churn_num(df)
df[['churn']].head()


,churn
0,0
1,0
2,0
3,1
4,1


In [6]:
def tenure_year(dataframe):
    dataframe['tenure_year'] = (dataframe.tenure / 12).astype(int) + 1
    return dataframe
df = tenure_year(df)
df[['tenure', 'tenure_year']].head()

,tenure,tenure_year
0,9,1
1,9,1
2,7,1
3,1,1
4,1,1


In [7]:
# for col in df.drop(columns=['customer_id']):
#     print(df[col].value_counts(dropna=False))

In [8]:
encoder = LabelEncoder()
encoder.fit(df.gender)
df.embarked = encoder.transform(df.gender)
df['gender_encode'] = encoder.transform(df.gender)
df[['gender_encode']].head()

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


,gender_encode
0,1
1,0
2,0
3,1
4,0


In [9]:
# 11: Numeric Scaling: scale the monthly_charges and total_charges data. 
# Make sure that the parameters for scaling are learned from the training data set.

# ~~~~~SPLIT~~~~~
# ~~This is also #9 ~~
# X = df.drop(['~~~~*INSERT COLUMNS*~~~~'], axis = 1)
# y = df[['churn']]
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.70, random_state=123)

# train = pd.concat([X_train, y_train], axis=1)
# test = pd.concat([X_test, y_test], axis=1)

# scaler = MinMaxScaler()
# scaler.fit(train[['monthly_charges', 'total_charges']])

# train[['monthly_charges', 'total_charges']] = scaler.transform(train[['monthly_charges', 'total_charges']])
# test[['monthly_charges', 'total_charges']] = scaler.transform(test[['monthly_charges', 'total_charges']])


In [10]:
# 10: Variable Encoding: encode the values in each non-numeric 
# feature such that they are numeric.

In [11]:
# 5 : Figure out a way to capture the information contained
# in phone_service and multiple_lines into a single variable of dtype int. 
# Write a function that will transform the data and place in a new column named phone_id.

df_service_lines = df[['phone_service', 'multiple_lines']]

df_service_lines['multiple_lines'] = df_service_lines['multiple_lines'].map({'No phone service': 'No', 'Yes': 'Yes', 'No': 'No'})
df_service_lines.head(50)


conditions_lns = [
    (df_service_lines['phone_service'] == 'Yes') & (df_service_lines['multiple_lines'] == 'Yes'),
    (df_service_lines['phone_service'] == 'Yes') & (df_service_lines['multiple_lines'] == 'No'),
    (df_service_lines['phone_service'] == 'No') & (df_service_lines['multiple_lines'] == 'No')
]
choices_lns = [2, 1, 0]
df_service_lines['phone_lines'] = np.select(conditions_lns, choices_lns)
df_service_lines.head(20)


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,phone_service,multiple_lines,phone_lines
0,Yes,Yes,2
1,Yes,No,1
2,Yes,No,1
3,No,No,0
4,No,No,0
5,Yes,No,1
6,No,No,0
7,Yes,No,1
8,Yes,Yes,2
9,Yes,Yes,2


In [12]:
# 6: Figure out a way to capture the information contained 
# in dependents and partner into a single variable of dtype int. 
# Transform the data and place in a new column household_type_id.

df_pdep = df[['partner', 'dependents']]
conditions = [
    (df_pdep['partner'] == 'Yes') & (df_pdep['dependents'] == 'Yes'),
    (df_pdep['partner'] == 'No') & (df_pdep['dependents'] == 'Yes'), 
    (df_pdep['partner'] == 'Yes') & (df_pdep['dependents'] == 'No'),
    (df_pdep['partner'] == 'No') & (df_pdep['dependents'] == 'No')
]
choices = [3, 2, 1, 0]
df_pdep['partner_dependents'] = np.select(conditions, choices)
df_pdep.head(20)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,partner,dependents,partner_dependents
0,No,No,0
1,No,Yes,2
2,No,No,0
3,No,No,0
4,Yes,Yes,3
5,No,No,0
6,No,Yes,2
7,No,No,0
8,Yes,No,1
9,No,No,0


In [13]:
# 7:
#Figure out a way to capture the information contained 
# in streaming_tv and streaming_movies into a single variable of dtype int. 
#Transform the data and place in a new column streaming_services.
# 0: no streaming tv or movies
# 1: streaming tv but not movies
# 2: streaming movies but not tv
# 3: streaming both tv and movies
df_streaming = df[['streaming_tv', 'streaming_movies']]
conditions_strm = [
    (df_streaming['streaming_tv'] == 'Yes') & (df_streaming['streaming_movies'] == 'Yes'),
    (df_streaming['streaming_tv'] == 'No') & (df_streaming['streaming_movies'] == 'Yes'), 
    (df_streaming['streaming_tv'] == 'Yes') & (df_streaming['streaming_movies'] == 'No'),
    (df_streaming['streaming_tv'] == 'No') & (df_streaming['streaming_movies'] == 'No')
]
choices = [3, 2, 1, 0]
df_streaming['streaming_services'] = np.select(conditions_strm, choices)
df_streaming.head(20)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,streaming_tv,streaming_movies,streaming_services
0,No,Yes,2
1,Yes,Yes,3
2,No,No,0
3,No,No,0
4,No,No,0
5,No,No,0
6,No,No,0
7,No,No,0
8,No,No,0
9,No,Yes,2


In [14]:
# 8:
# Figure out a way to capture the information contained in 
# online_security and online_backup into a single variable of dtype int. 
# Transform the data and place in a new column online_security_backup.
df_secback = df[['online_security', 'online_backup']]
df_secback.head()
# 0: no security or backup
# 1: online security but no backup
# 2: backup but no security
# 3: both online security and backup
conditions_secback = [
    (df_secback['online_security'] == 'Yes') & (df_secback['online_backup'] == 'Yes'),
    (df_secback['online_security'] == 'No') & (df_secback['online_backup'] == 'Yes'), 
    (df_secback['online_security'] == 'Yes') & (df_secback['online_backup'] == 'No'),
    (df_secback['online_security'] == 'No') & (df_secback['online_backup'] == 'No')
]
choices = [3, 2, 1, 0]
df_secback['secback'] = np.select(conditions_secback, choices)
df_secback.head(20)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,online_security,online_backup,secback
0,No,No,0
1,No,No,0
2,Yes,No,1
3,No,No,0
4,Yes,No,1
5,Yes,No,1
6,Yes,Yes,3
7,No,No,0
8,No,No,0
9,Yes,No,1


In [15]:
def encode_gender(df):
    '''encodes gender'''
    encoder = LabelEncoder()
    encoder.fit(df.gender)
    return df.assign(gender_e = encoder.transform(df.gender))



In [16]:
df = encode_gender(df)
df[['gender', 'gender_e']].head()

,gender,gender_e
0,Male,1
1,Female,0
2,Female,0
3,Male,1
4,Female,0


In [17]:
def encode_seniors(df):
    encoder=LabelEncoder()
    encoder.fit(df.senior_citizen)
    return df.assign(senior_citizen_e = encoder.transform(df.senior_citizen))



In [18]:
df = encode_seniors(df)
df[['senior_citizen', 'senior_citizen_e']].head()

,senior_citizen,senior_citizen_e
0,0,0
1,0,0
2,1,1
3,1,1
4,0,0


In [19]:
for col in df:
    print(col)

customer_id
gender
senior_citizen
partner
dependents
tenure
phone_service
multiple_lines
internet_service_type_id
online_security
online_backup
device_protection
tech_support
streaming_tv
streaming_movies
contract_type_id
paperless_billing
payment_type_id
monthly_charges
total_charges
churn
contract_type
internet_service_type
payment_type
num_lines
partner_dependents
streaming_services
secback
tenure_year
gender_encode
gender_e
senior_citizen_e


In [21]:
df[['tech_support']].head()

,tech_support
0,No
1,Yes
2,No
3,No
4,No
